In [ ]:
import requests
import json
import time

In [ ]:
# url_da_pesquisa = "https://www.ebi.ac.uk/metagenomics/api/v1/biomes/root:Environmental:Aquatic:Marine/genomes"
# url_da_pesquisa = "https://www.ebi.ac.uk/metagenomics/api/v1/genome-catalogues/marine-v2-0/genomes"

def pesquisa_MGnify(max_pages):
    """
    Faz a pesquisa de vários IDs de amostras do MGnify. A pesquisa pode ser feita pelo catálogo de genomas do MGnify ou por bioma.

    Parâmetros:
    max_pages: número máximo de páginas a buscar.
    """
    lista_ids = []

    for page in range(1, max_pages + 1):
        url = f"https://www.ebi.ac.uk/metagenomics/api/v1/biomes/root:Environmental:Aquatic:Marine/genomes?page={page}"
        resposta = requests.get(url)

        if resposta.status_code != 200:
            print(f"Erro {resposta.status_code} na página {page}")
            break

        dados_json = resposta.json()
        ids_atuais = [item["id"] for item in dados_json.get("data", [])]
        lista_ids.extend(ids_atuais)

        print(f"Página {page}: {len(ids_atuais)} IDs encontrados")
        time.sleep(1)

    with open("aquatic_download.json", "w") as i:
        json.dump(dados_json, i, indent=4)

    with open("aquatic_ids.txt", "w") as f:
        for id_study in lista_ids:
            f.write(id_study + "\n")

    print(f"\nTotal de {len(lista_ids)} IDs coletados em {page} páginas.")
    return "aquatic_download.json", lista_ids

In [ ]:
arquivo, lista_ids = pesquisa_MGnify(5)

In [ ]:
print(lista_ids)

In [ ]:
def genome_list_url(lista_ids, arquivo_saida="links_download_genomes.txt"):
    '''
    Faz a pesquisa das urls associadas com cada uma das amostras. Junta todas as urls em um txt único.

    Parâmetros:
    lista_ids: lista com os IDs das amostras. Ex: 'MGYG000296006'
    arquivo_saida: nome do arquivo de saída onde as urls serão salvas. Default: 'links_download_genomes.txt'
    '''
    lista_download = []
    for ids in lista_ids:
        print(f"Baixando links para o ID {ids}")
        url = f"https://www.ebi.ac.uk/metagenomics/api/v1/genomes/{ids}/downloads"
        resposta =requests.get(url)
        print(resposta.status_code)
        print(resposta.text[:1000])
        if resposta.status_code == 200:
            url_ids = [item["links"]["self"] for item in resposta.json().get("data", [])]
            lista_download.extend(url_ids)
        else:
            print("Erro na requisição.")
    with open(arquivo_saida, "w") as f:
        for link in lista_download:
            f.write(link + "\n")
    print (lista_download)
    return lista_download

In [ ]:
lista_links = genome_list_url(lista_ids)

In [ ]:
print(lista_links)

In [ ]:
def download_genome_url(entrada, filtro=".fna"):
    """
    Lê um arquivo .txt OU uma lista de links e baixa apenas os arquivos que você quer filtrando pelo tipo de arquivo.

    Parâmetros:
    entrada: caminho do arquivo .txt contendo os links OU uma lista de links.
    filtro: extensão do arquivo a ser baixado. Default: ".fna"
    """
    if isinstance(entrada, str):
        with open(entrada, "r") as f:
            links = [linha.strip() for linha in f if linha.strip()]
    elif isinstance(entrada, list):
        links = [linha.strip() for linha in entrada if linha.strip()]
    else:
        raise TypeError("A entrada deve ser um caminho de arquivo (.txt) ou uma lista de links.")

    fna_links = [link for link in links if link.endswith(filtro)]

    if not fna_links:
        print(f"Nenhum link com {filtro} encontrado.")
        return

    for link in fna_links:
        nome_arquivo = link.split("/")[-1]
        print(f"Baixando {nome_arquivo} ...")

        try:
            resposta = requests.get(link, timeout=30)
            resposta.raise_for_status()
            with open(nome_arquivo, "wb") as f:
                f.write(resposta.content)
            print(f"Download concluído: {nome_arquivo}")
        except requests.RequestException as e:
            print(f"Erro ao baixar {nome_arquivo}: {e}")

In [ ]:
fna_teste = download_genome_url(lista_links)